调参方法
* offks + 0.8(offks - devks)最大化  
* (devks+offks)/2 最大化 
* offks最大化  

这里只举例offks + 0.8(offks - devks)最大化

In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,auc
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import numpy as np
import random
import math
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('Acard.txt')
data.head()

,obs_mth,bad_ind,uid,td_score,jxl_score,mj_score,rh_score,zzc_score,zcx_score,person_info,finance_info,credit_info,act_info
0,2018-10-31,0.0,A10000005,0.675349,0.144072,0.186899,0.483640,0.928328,0.369644,-0.322581,0.023810,0.00,0.217949
1,2018-07-31,0.0,A1000002,0.825269,0.398688,0.139396,0.843725,0.605194,0.406122,-0.128677,0.023810,0.00,0.423077
2,2018-09-30,0.0,A1000011,0.315406,0.629745,0.535854,0.197392,0.614416,0.320731,0.062660,0.023810,0.10,0.448718
3,2018-07-31,0.0,A10000481,0.002386,0.609360,0.366081,0.342243,0.870006,0.288692,0.078853,0.071429,0.05,0.179487
4,2018-07-31,0.0,A1000069,0.406310,0.405352,0.783015,0.563953,0.715454,0.512554,-0.261014,0.023810,0.00,0.423077


In [3]:
train = data[data.obs_mth != '2018-11-30'].reset_index().copy()
val = data[data.obs_mth == '2018-11-30'].reset_index().copy()
feature_lst = ['person_info','finance_info','credit_info','act_info']
x = train[feature_lst]
y = train['bad_ind']

val_x =  val[feature_lst]
val_y = val['bad_ind']


train_x,test_x,train_y,test_y = train_test_split(x,y,random_state=0,test_size=0.2)

In [20]:
#改变我们想去调整的参数为value，设置调参区间
min_value = 40
max_value = 60
for value in  range(min_value,max_value+1):
    best_omd = -1
    best_value = -1
    best_ks=[]
    def  lgb_test(train_x,train_y,test_x,test_y):
        clf =lgb.LGBMClassifier(boosting_type = 'gbdt',
                               objective = 'binary',
                               metric = 'auc',
                               learning_rate = 0.1,
                               n_estimators = value,
                               max_depth = 5,
                               num_leaves = 20,
                               max_bin = 45,
                               min_data_in_leaf = 6,
                               bagging_fraction = 0.6,
                               bagging_freq = 0,
                               feature_fraction = 0.8,
                               silent=True
                               )
        clf.fit(train_x,train_y,eval_set = [(train_x,train_y),(test_x,test_y)],eval_metric = 'auc')
        return clf,clf.best_score_['valid_1']['auc'],
    lgb_model , lgb_auc  = lgb_test(train_x,train_y,test_x,test_y)

    y_pred = lgb_model.predict_proba(x)[:,1]
    fpr_lgb_train,tpr_lgb_train,_ = roc_curve(y,y_pred)
    train_ks = abs(fpr_lgb_train - tpr_lgb_train).max()

    y_pred = lgb_model.predict_proba(val_x)[:,1]
    fpr_lgb,tpr_lgb,_ = roc_curve(val_y,y_pred)
    val_ks = abs(fpr_lgb - tpr_lgb).max()
    
    Omd= val_ks + 0.8*(val_ks - train_ks)
    if Omd>best_omd:
        best_omd = Omd
        best_value = value
        best_ks = [train_ks,val_ks]
print('best_value:',best_value)
print('best_ks:',best_ks)

[1]	training's auc: 0.759109	valid_1's auc: 0.754548
[2]	training's auc: 0.80654	valid_1's auc: 0.805116
[3]	training's auc: 0.812436	valid_1's auc: 0.810926
[4]	training's auc: 0.818706	valid_1's auc: 0.814813
[5]	training's auc: 0.819127	valid_1's auc: 0.815716
[6]	training's auc: 0.820571	valid_1's auc: 0.816072
[7]	training's auc: 0.821937	valid_1's auc: 0.814158
[8]	training's auc: 0.822095	valid_1's auc: 0.815659
[9]	training's auc: 0.822149	valid_1's auc: 0.815565
[10]	training's auc: 0.822313	valid_1's auc: 0.815155
[11]	training's auc: 0.821869	valid_1's auc: 0.813779
[12]	training's auc: 0.823628	valid_1's auc: 0.815481
[13]	training's auc: 0.824727	valid_1's auc: 0.81524
[14]	training's auc: 0.824859	valid_1's auc: 0.815608
[15]	training's auc: 0.825706	valid_1's auc: 0.816611
[16]	training's auc: 0.826626	valid_1's auc: 0.816357
[17]	training's auc: 0.827403	valid_1's auc: 0.816072
[18]	training's auc: 0.827806	valid_1's auc: 0.815606
[19]	training's auc: 0.82813	valid_1's 

[34]	training's auc: 0.834511	valid_1's auc: 0.816331
[35]	training's auc: 0.834785	valid_1's auc: 0.816389
[36]	training's auc: 0.835309	valid_1's auc: 0.815987
[37]	training's auc: 0.835563	valid_1's auc: 0.815713
[38]	training's auc: 0.835988	valid_1's auc: 0.815741
[39]	training's auc: 0.83654	valid_1's auc: 0.815597
[40]	training's auc: 0.836964	valid_1's auc: 0.815413
[41]	training's auc: 0.837136	valid_1's auc: 0.815315
[42]	training's auc: 0.837233	valid_1's auc: 0.815039
[43]	training's auc: 0.8374	valid_1's auc: 0.814683
[44]	training's auc: 0.837624	valid_1's auc: 0.815244
[45]	training's auc: 0.837793	valid_1's auc: 0.815338
[46]	training's auc: 0.838263	valid_1's auc: 0.814794
[47]	training's auc: 0.838458	valid_1's auc: 0.81489
[48]	training's auc: 0.838852	valid_1's auc: 0.81475
[49]	training's auc: 0.839178	valid_1's auc: 0.814698
[50]	training's auc: 0.839295	valid_1's auc: 0.814591
[51]	training's auc: 0.839608	valid_1's auc: 0.81442
[52]	training's auc: 0.839626	vali

[32]	training's auc: 0.833423	valid_1's auc: 0.816643
[33]	training's auc: 0.833927	valid_1's auc: 0.816141
[34]	training's auc: 0.834511	valid_1's auc: 0.816331
[35]	training's auc: 0.834785	valid_1's auc: 0.816389
[36]	training's auc: 0.835309	valid_1's auc: 0.815987
[37]	training's auc: 0.835563	valid_1's auc: 0.815713
[38]	training's auc: 0.835988	valid_1's auc: 0.815741
[39]	training's auc: 0.83654	valid_1's auc: 0.815597
[40]	training's auc: 0.836964	valid_1's auc: 0.815413
[41]	training's auc: 0.837136	valid_1's auc: 0.815315
[42]	training's auc: 0.837233	valid_1's auc: 0.815039
[43]	training's auc: 0.8374	valid_1's auc: 0.814683
[44]	training's auc: 0.837624	valid_1's auc: 0.815244
[45]	training's auc: 0.837793	valid_1's auc: 0.815338
[46]	training's auc: 0.838263	valid_1's auc: 0.814794
[47]	training's auc: 0.838458	valid_1's auc: 0.81489
[48]	training's auc: 0.838852	valid_1's auc: 0.81475
[49]	training's auc: 0.839178	valid_1's auc: 0.814698
[50]	training's auc: 0.839295	val

[36]	training's auc: 0.835309	valid_1's auc: 0.815987
[37]	training's auc: 0.835563	valid_1's auc: 0.815713
[38]	training's auc: 0.835988	valid_1's auc: 0.815741
[39]	training's auc: 0.83654	valid_1's auc: 0.815597
[40]	training's auc: 0.836964	valid_1's auc: 0.815413
[41]	training's auc: 0.837136	valid_1's auc: 0.815315
[42]	training's auc: 0.837233	valid_1's auc: 0.815039
[43]	training's auc: 0.8374	valid_1's auc: 0.814683
[44]	training's auc: 0.837624	valid_1's auc: 0.815244
[45]	training's auc: 0.837793	valid_1's auc: 0.815338
[46]	training's auc: 0.838263	valid_1's auc: 0.814794
[47]	training's auc: 0.838458	valid_1's auc: 0.81489
[48]	training's auc: 0.838852	valid_1's auc: 0.81475
[49]	training's auc: 0.839178	valid_1's auc: 0.814698
[50]	training's auc: 0.839295	valid_1's auc: 0.814591
[51]	training's auc: 0.839608	valid_1's auc: 0.81442
[52]	training's auc: 0.839626	valid_1's auc: 0.814295
[53]	training's auc: 0.839704	valid_1's auc: 0.814093
[54]	training's auc: 0.839753	vali

[34]	training's auc: 0.834511	valid_1's auc: 0.816331
[35]	training's auc: 0.834785	valid_1's auc: 0.816389
[36]	training's auc: 0.835309	valid_1's auc: 0.815987
[37]	training's auc: 0.835563	valid_1's auc: 0.815713
[38]	training's auc: 0.835988	valid_1's auc: 0.815741
[39]	training's auc: 0.83654	valid_1's auc: 0.815597
[40]	training's auc: 0.836964	valid_1's auc: 0.815413
[41]	training's auc: 0.837136	valid_1's auc: 0.815315
[42]	training's auc: 0.837233	valid_1's auc: 0.815039
[43]	training's auc: 0.8374	valid_1's auc: 0.814683
[44]	training's auc: 0.837624	valid_1's auc: 0.815244
[45]	training's auc: 0.837793	valid_1's auc: 0.815338
[46]	training's auc: 0.838263	valid_1's auc: 0.814794
[47]	training's auc: 0.838458	valid_1's auc: 0.81489
[48]	training's auc: 0.838852	valid_1's auc: 0.81475
[49]	training's auc: 0.839178	valid_1's auc: 0.814698
[50]	training's auc: 0.839295	valid_1's auc: 0.814591
[51]	training's auc: 0.839608	valid_1's auc: 0.81442
[52]	training's auc: 0.839626	vali

...

[59]	training's auc: 0.84062	valid_1's auc: 0.813642
[60]	training's auc: 0.841004	valid_1's auc: 0.813584
[61]	training's auc: 0.841111	valid_1's auc: 0.813572
[62]	training's auc: 0.841184	valid_1's auc: 0.813624
[63]	training's auc: 0.841392	valid_1's auc: 0.813496
[64]	training's auc: 0.84156	valid_1's auc: 0.813174
[65]	training's auc: 0.841855	valid_1's auc: 0.813039
[66]	training's auc: 0.841921	valid_1's auc: 0.813021
[67]	training's auc: 0.84206	valid_1's auc: 0.813137
[68]	training's auc: 0.842272	valid_1's auc: 0.813186
[69]	training's auc: 0.842458	valid_1's auc: 0.81305
[70]	training's auc: 0.842688	valid_1's auc: 0.812753
[71]	training's auc: 0.842889	valid_1's auc: 0.81302
[72]	training's auc: 0.843057	valid_1's auc: 0.812872
[73]	training's auc: 0.843109	valid_1's auc: 0.812855
[74]	training's auc: 0.843262	valid_1's auc: 0.812789
[75]	training's auc: 0.843449	valid_1's auc: 0.812805
[76]	training's auc: 0.84353	valid_1's auc: 0.812675
[77]	training's auc: 0.843629	vali

[31]	training's auc: 0.833222	valid_1's auc: 0.816553
[32]	training's auc: 0.833423	valid_1's auc: 0.816643
[33]	training's auc: 0.833927	valid_1's auc: 0.816141
[34]	training's auc: 0.834511	valid_1's auc: 0.816331
[35]	training's auc: 0.834785	valid_1's auc: 0.816389
[36]	training's auc: 0.835309	valid_1's auc: 0.815987
[37]	training's auc: 0.835563	valid_1's auc: 0.815713
[38]	training's auc: 0.835988	valid_1's auc: 0.815741
[39]	training's auc: 0.83654	valid_1's auc: 0.815597
[40]	training's auc: 0.836964	valid_1's auc: 0.815413
[41]	training's auc: 0.837136	valid_1's auc: 0.815315
[42]	training's auc: 0.837233	valid_1's auc: 0.815039
[43]	training's auc: 0.8374	valid_1's auc: 0.814683
[44]	training's auc: 0.837624	valid_1's auc: 0.815244
[45]	training's auc: 0.837793	valid_1's auc: 0.815338
[46]	training's auc: 0.838263	valid_1's auc: 0.814794
[47]	training's auc: 0.838458	valid_1's auc: 0.81489
[48]	training's auc: 0.838852	valid_1's auc: 0.81475
[49]	training's auc: 0.839178	val

[1]	training's auc: 0.759109	valid_1's auc: 0.754548
[2]	training's auc: 0.80654	valid_1's auc: 0.805116
[3]	training's auc: 0.812436	valid_1's auc: 0.810926
[4]	training's auc: 0.818706	valid_1's auc: 0.814813
[5]	training's auc: 0.819127	valid_1's auc: 0.815716
[6]	training's auc: 0.820571	valid_1's auc: 0.816072
[7]	training's auc: 0.821937	valid_1's auc: 0.814158
[8]	training's auc: 0.822095	valid_1's auc: 0.815659
[9]	training's auc: 0.822149	valid_1's auc: 0.815565
[10]	training's auc: 0.822313	valid_1's auc: 0.815155
[11]	training's auc: 0.821869	valid_1's auc: 0.813779
[12]	training's auc: 0.823628	valid_1's auc: 0.815481
[13]	training's auc: 0.824727	valid_1's auc: 0.81524
[14]	training's auc: 0.824859	valid_1's auc: 0.815608
[15]	training's auc: 0.825706	valid_1's auc: 0.816611
[16]	training's auc: 0.826626	valid_1's auc: 0.816357
[17]	training's auc: 0.827403	valid_1's auc: 0.816072
[18]	training's auc: 0.827806	valid_1's auc: 0.815606
[19]	training's auc: 0.82813	valid_1's 

[85]	training's auc: 0.845192	valid_1's auc: 0.812129
[86]	training's auc: 0.845408	valid_1's auc: 0.811908
[87]	training's auc: 0.84552	valid_1's auc: 0.811538
[88]	training's auc: 0.845829	valid_1's auc: 0.811148
[89]	training's auc: 0.845955	valid_1's auc: 0.811247
[90]	training's auc: 0.84609	valid_1's auc: 0.810841
[91]	training's auc: 0.84622	valid_1's auc: 0.810721
[92]	training's auc: 0.846426	valid_1's auc: 0.81055
[93]	training's auc: 0.846501	valid_1's auc: 0.81046
[94]	training's auc: 0.846683	valid_1's auc: 0.810514
[95]	training's auc: 0.84688	valid_1's auc: 0.811042
[96]	training's auc: 0.847005	valid_1's auc: 0.810996
[1]	training's auc: 0.759109	valid_1's auc: 0.754548
[2]	training's auc: 0.80654	valid_1's auc: 0.805116
[3]	training's auc: 0.812436	valid_1's auc: 0.810926
[4]	training's auc: 0.818706	valid_1's auc: 0.814813
[5]	training's auc: 0.819127	valid_1's auc: 0.815716
[6]	training's auc: 0.820571	valid_1's auc: 0.816072
[7]	training's auc: 0.821937	valid_1's au

[54]	training's auc: 0.839753	valid_1's auc: 0.814177
[55]	training's auc: 0.840068	valid_1's auc: 0.814037
[56]	training's auc: 0.840172	valid_1's auc: 0.813711
[57]	training's auc: 0.84037	valid_1's auc: 0.814101
[58]	training's auc: 0.840536	valid_1's auc: 0.814035
[59]	training's auc: 0.84062	valid_1's auc: 0.813642
[60]	training's auc: 0.841004	valid_1's auc: 0.813584
[61]	training's auc: 0.841111	valid_1's auc: 0.813572
[62]	training's auc: 0.841184	valid_1's auc: 0.813624
[63]	training's auc: 0.841392	valid_1's auc: 0.813496
[64]	training's auc: 0.84156	valid_1's auc: 0.813174
[65]	training's auc: 0.841855	valid_1's auc: 0.813039
[66]	training's auc: 0.841921	valid_1's auc: 0.813021
[67]	training's auc: 0.84206	valid_1's auc: 0.813137
[68]	training's auc: 0.842272	valid_1's auc: 0.813186
[69]	training's auc: 0.842458	valid_1's auc: 0.81305
[70]	training's auc: 0.842688	valid_1's auc: 0.812753
[71]	training's auc: 0.842889	valid_1's auc: 0.81302
[72]	training's auc: 0.843057	vali